# Initiation

In [1]:
%load_ext autoreload
%autoreload 2

import os
import random
import pprint

import pysam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_packages')

In [2]:
from handygenome.common import ChromDict, Vcfspec, Interval
from handygenome.variantplus.breakends import Breakends
from handygenome.variantplus.variantplus import VariantPlus
from handygenome.variantplus.vcfplus import VcfPlus
from handygenome import igvhandle

In [3]:
FASTA_PATH_HG19 = "/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.fasta"
FASTA_PATH_HG38 = "/home/users/data/01_reference/human_g1k_v38/Homo_sapiens_assembly38.fasta"

FASTA_HG19 = pysam.FastaFile(FASTA_PATH_HG19)
FASTA_HG38 = pysam.FastaFile(FASTA_PATH_HG38)
CHROMDICT_HG19 = ChromDict(fasta_path=FASTA_PATH_HG19)
CHROMDICT_HG38 = ChromDict(fasta_path=FASTA_PATH_HG38)

In [4]:
igv = igvhandle.IGVHandle(60387)

# Scratches

In [7]:
arr = np.array([np.nan, 1, 2, 3])

In [16]:
(arr >= 2).sum()

2

In [5]:
from handygenome.annotation.readstats import get_readstats

In [6]:
tbam_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/IO17/LU-IO17.tumor.bam'
nbam_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/IO17/LU-IO17.normal.bam'

In [7]:
tbam = pysam.AlignmentFile(tbam_path)
nbam = pysam.AlignmentFile(nbam_path)

In [8]:
vcfspec = Vcfspec('7', 158254690, 'C', ('A',))

In [26]:
vcfspec

<Vcfspec (7:158254690 C>A)>

In [13]:
igv.load([tbam_path, nbam_path])
igv.goto([vcfspec])

OK
OK
OK


In [33]:
# for read in tbam.fetch(vcfspec.chrom, vcfspec.pos0, vcfspec.end0):
for read in tbam.fetch('7', 158_254_656 - 4, 158_254_656 + 4):
#     if read.query_name == 'A00125:179:HFVCKDSXX:1:1676:11794:4883' and read.reference_start == 158254690 - 1:
#     if read.query_name == 'A00125:179:HFVCKDSXX:4:1534:14841:26349':
    if (read.query_name == 'A00125:179:HFVCKDSXX:3:1343:3269:7842' and read.cigartuples[0][0] == 4 and read.cigartuples[-1][0] == 4):
        break

In [35]:
print(read.query_name)
print(read.reference_start)
print(read.reference_end)
print(read.cigarstring)

A00125:179:HFVCKDSXX:3:1343:3269:7842
158254655
158254656
23S108I20S


In [74]:
readstats = get_readstats(vcfspec, tbam, FASTA_HG19, CHROMDICT_HG19, no_matesearch=True)

In [79]:
vcf_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/09_short_variants_union/220707_indel/02_union/panel_indel_union.annot.readstats_again.vcf.gz'

In [80]:
vcf = pysam.VariantFile(vcf_path)

In [81]:
vr = next(vcf.fetch())

In [92]:
vp = VariantPlus(vr)

In [93]:
vp.readstats_dict['LU-14_panel']

{'mean_BQs': {None: nan, -1: 28.75, 0: 27.678571428571427, 1: nan},
 'mean_MQs': {None: 40.142857142857146, -1: 48.5, 0: 36.25, 1: nan},
 'mean_cliplens': {None: 2.0, -1: 0.0, 0: 0.6530612244897959, 1: nan},
 'mean_varpos_fractions': {None: nan,
                           -1: 0.6372185430463576,
                           0: 0.29559487901533404,
                           1: nan},
 'median_BQs': {None: nan, -1: 32.5, 0: 26.0, 1: nan},
 'median_MQs': {None: 50.0, -1: 48.5, 0: 27.0, 1: nan},
 'median_cliplens': {None: 0.0, -1: 0.0, 0: 0.0, 1: nan},
 'median_varpos_fractions': {None: nan,
                             -1: 0.6372185430463576,
                             0: 0.21593818984547464,
                             1: nan},
 'rppcounts': {None: 7, -1: 2, 0: 98, 1: 0, 'softclip_overlap': 1},
 'total_rppcount': 108,
 'varpos_uniform_pvalues': {None: nan,
                            -1: 0.7112,
                            0: 8.356399858482668e-14,
                            1: nan}}

In [103]:
vp.annotdb.popfreq

{'1000Genomes': None,
 'ALSPAC': None,
 'Chileans': None,
 'ChromosomeY': None,
 'Daghestan': None,
 'Estonian': None,
 'ExAC': None,
 'FINRISK': None,
 'GENOME_DK': None,
 'GnomAD': None,
 'GnomAD_exomes': None,
 'GoESP': None,
 'GoNL': None,
 'HGDP_Stanford': None,
 'HapMap': None,
 'KOREAN': None,
 'Korea1K': None,
 'MGP': None,
 'NorthernSweden': None,
 'PAGE_STUDY': None,
 'PRJEB36033': None,
 'PRJEB37584': None,
 'PRJEB37766': None,
 'PharmGKB': None,
 'Qatari': None,
 'SGDP_PRJ': None,
 'Siberian': None,
 'TOMMO': None,
 'TOPMED': None,
 'TWINSUK': None,
 'Vietnamese': None,
 'dbGaP_PopFreq': None}

In [119]:
from handygenome.variantplus.vpfilter import PopfreqFilter, ReadcountFilter

In [120]:
fi1 = PopfreqFilter()
fi2 = ReadcountFilter()
fi3 = 

In [117]:
fi1.check(vp)

True

In [125]:
fi2.check(vp, 'LU-14_panel')

False

In [96]:
from handygenome.annotation.readstats import get_readstats
bam = pysam.AlignmentFile('/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/14/LU-14.panel.bam')
readstats = get_readstats(vp.vcfspec, bam, FASTA_HG19, CHROMDICT_HG19, no_matesearch=True)

NDX550168:116:HH35KBGXC:4:13507:20047:10939


/home/users/pjh/tools/miniconda/210821/miniconda3/envs/genome_v5/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/users/pjh/tools/miniconda/210821/miniconda3/envs/genome_v5/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [88]:
igv.load(['/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/14/LU-14.panel.bam'])

OK


In [89]:
igv.goto([vp.vcfspec])

OK


In [98]:
vp.vcfspec

<Vcfspec (1:1580929 GA>G)>

In [99]:
for read in bam.fetch('1', 1580929 - 100, 1580929 + 100):
    if read.query_name == 'NDX550168:116:HH35KBGXC:4:13507:20047:10939':
        print(read)

NDX550168:116:HH35KBGXC:4:13507:20047:10939	83	#0	1580793	27	114M1I22M14S	#0	1580590	-339	CGGTGGCGCGATCTCGGCTCACTGCAACCTCCGCCTCCCGGGTTCGAGCGATTCTCCTGCCTCTGCCTCCCGAGTAGCTGGGATTACAGGCGTGCGCCCTGTACTCCCAGCTAATTTTTTGTATTTTTTGTAGAGACAAGGTTTTGCCATG	array('B', [8, 25, 25, 28, 31, 28, 26, 37, 26, 36, 28, 12, 33, 35, 26, 33, 37, 33, 26, 33, 34, 33, 36, 37, 32, 36, 13, 33, 30, 29, 32, 8, 36, 32, 30, 34, 30, 30, 22, 13, 35, 35, 34, 34, 25, 32, 35, 31, 24, 32, 26, 13, 12, 32, 28, 32, 28, 34, 22, 31, 29, 27, 29, 11, 36, 28, 31, 32, 32, 31, 25, 35, 34, 35, 30, 33, 11, 31, 34, 34, 34, 34, 34, 34, 27, 27, 28, 36, 24, 11, 9, 35, 12, 36, 21, 11, 32, 31, 31, 34, 34, 32, 32, 28, 33, 31, 29, 32, 31, 36, 29, 32, 15, 29, 33, 27, 34, 33, 33, 34, 34, 31, 35, 34, 15, 34, 33, 27, 35, 35, 32, 35, 35, 36, 35, 32, 32, 35, 35, 35, 35, 34, 35, 28, 35, 36, 34, 33, 30, 31, 31])	[('BD', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN